In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Test').getOrCreate() 

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/04 16:48:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Set file
logFile = '/notebooks/airflow.cfg'
# Read file
logData = spark.sparkContext.textFile(logFile).cache()
# Get lines with A
numAs = logData.filter(lambda s: 'a' in s).count()
# Get lines with B 
numBs = logData.filter(lambda s: 'b' in s).count()
# Print result
print('Lines with a: {}, lines with b: {}'.format(numAs, numBs))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/notebooks/airflow.cfg
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:205)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:300)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:296)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:300)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:296)
	at org.apache.spark.api.python.PythonRDD.getPa

In [3]:
!cd $SPARK_HOME/jars && wget https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar

--2022-04-01 17:08:04--  https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.30.208, 172.217.28.112, 142.250.78.176, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.30.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31607894 (30M) [application/java-archive]
gcs-connector-hadoop3-latest.jar: Permission denied

Cannot write to ‘gcs-connector-hadoop3-latest.jar’ (Success).


In [5]:
from pyspark.sql import SparkSession
import os

spark = SparkSession \
        .builder \
        .appName('Access GCS') \
        .config("spark.jars", "work/jars/gcs-connector-hadoop3-latest.jar") \
        .getOrCreate()

spark._jsc.hadoopConfiguration() \
    .set("google.cloud.auth.service.account.json.keyfile","/.google/credentials/google_credentials_project.json")


BUCKET = os.getenv('GCP_GCS_BUCKET')

#This path has to be dynamically defined for each table
def transform_parquet(table, file_date='*'):
    path="https://storage.googleapis.com/dtc_data_lake_de-stack-overflow/parquet/badges-2008-08.parquet"

    df = spark.read.parquet(path, header=True)
    df.show()


In [6]:
transform_parquet('badges', '2008-08')

Py4JJavaError: An error occurred while calling o61.parquet.
: java.lang.UnsupportedOperationException
	at org.apache.hadoop.fs.http.AbstractHttpFileSystem.listStatus(AbstractHttpFileSystem.java:94)
	at org.apache.hadoop.fs.http.HttpsFileSystem.listStatus(HttpsFileSystem.java:23)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(HadoopFSUtils.scala:225)
	at org.apache.spark.util.HadoopFSUtils$.$anonfun$parallelListLeafFilesInternal$1(HadoopFSUtils.scala:95)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:286)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at scala.collection.TraversableLike.map(TraversableLike.scala:286)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:279)
	at scala.collection.AbstractTraversable.map(Traversable.scala:108)
	at org.apache.spark.uti